In [ ]:
!pip install transformers[torch]

In [7]:
import transformers
from transformers import AutoTokenizer
from datasets import load_dataset

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, f1_score

import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [34]:
# Load the emotions data set
emotions = load_dataset("emotion", trust_remote_code=True)

# Tokenize the text data
tokenizer=AutoTokenizer.from_pretrained("distilbert-base-cased")
def tokenize(batch):
 return tokenizer(batch["text"], padding=True, truncation=True)

data_encoded= emotions.map(tokenize,batched=True, batch_size=None)
lbl2Id = data_encoded['train'].features['label']._str2int

In [8]:
# Functions to plot confusion matrix and compute performance metrics
def plot_confusion_matrix(y_true, y_predictions, labels, normalize=None ):
    cm = confusion_matrix(y_valid, y_pred,labels=labels,normalize=normalize)

    fig, ax = plt.subplots(figsize=(6,6))
    cm_disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(id2label.values()))
    cm_disp.plot(cmap='Blues',colorbar=False,values_format='.2f',ax=ax)
    plt.title("Confusion Matrix")
    plt.show()


def compute_metrics(EvalPrediction):
    labels = EvalPrediction.label_ids
    preds = EvalPrediction.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, "f1_score": f1}

## Fine tuning the transformer for classification

In [9]:
# loging into hugging face
from huggingface_hub import notebook_login
notebook_login()

In [39]:
# Load pretrained transformer
from transformers import AutoModelForSequenceClassification
num_labels = data_encoded['train'].features['label'].num_classes
model_ckpt = 'distilbert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)
model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
# Defining all the hyperparameters for training run
from transformers import TrainingArguments, Trainer
batch_size=64
logging_steps = len(emotions_encoded['train'])//batch_size
model_name="distilbert-base-uncased-finetuned-emotion"
# training arguments
training_args = TrainingArguments(output_dir=model_name,
                                 num_train_epochs=6,
                                 learning_rate=2e-5,
                                 per_device_train_batch_size=batch_size,
                                 per_device_eval_batch_size=batch_size,
                                 weight_decay=0.01,
                                 evaluation_strategy='epoch',
                                 disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=True,
                                  log_level='error'
                                 )

In [41]:
# Enable CUDA_LAUNCH_BLOCKING for more informative error messages
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
trainer = Trainer(model=model,
                  args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset = emotions_encoded["train"],
                  eval_dataset = emotions_encoded["validation"],
                  tokenizer=tokenizer
                  )
trainer.train()

Epoch,Training Loss,Validation Loss



KeyboardInterrupt


KeyboardInterrupt: 